In [1]:
#Running and buliding the recommender system on given dummy data

#Import the required libraries
import numpy as np
import pandas as pd

#Making dataframe with the given dataset
df = pd.DataFrame()

id =  [0,1,2,3]
website_name = ['Gitlab.com','Cloudbees.com','squarespace.com','Wix.com']
Service = ['From project planning and source code management to CI/CD and monitoring, GitLab is a complete DevOps platform, delivered as a single application. Only GitLab enables Concurrent DevOps to make the software lifecycle 200% faster','Reduce risk, optimize software delivery and accelerate innovation with CloudBees - the industry-leading DevOps technology platform. Build Stuff That Matters','Squarespace is the all-in-one solution for anyone looking to create a beautiful website. Domains, eCommerce, hosting, galleries, analytics, and 24/7 support all included','Create a free website with Wix.com. Choose a stunning template and customize anything with the Wix website builder—no coding skills needed. Create yours today!']

df['id'] = id
df['website'] = website_name
df['Service'] = Service

df


,id,website,Service
0,0,Gitlab.com,From project planning and source code manageme...
1,1,Cloudbees.com,"Reduce risk, optimize software delivery and ac..."
2,2,squarespace.com,Squarespace is the all-in-one solution for any...
3,3,Wix.com,Create a free website with Wix.com. Choose a s...


In [2]:
#Data cleaning steps
#Parsing the service column
from html.parser import HTMLParser
html_parser = HTMLParser()
df['Service'] = df['Service'].apply(lambda x: html_parser.unescape(x))
df


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  """


,id,website,Service
0,0,Gitlab.com,From project planning and source code manageme...
1,1,Cloudbees.com,"Reduce risk, optimize software delivery and ac..."
2,2,squarespace.com,Squarespace is the all-in-one solution for any...
3,3,Wix.com,Create a free website with Wix.com. Choose a s...


In [3]:

# Appostophes = {"'s":"is","'re":"are"}
# words = df['Service']).split()
# df['Service'] = [Appostophes[word] if word in Appostophes else word for word in words]
# df['Service'] = "".join(df['Service'])

#Lowering all the words
import re
df['Service']= df['Service'].apply(lambda x: x.lower())

#Extracting only words from Service
df['Service'] = df['Service'].apply(lambda x: re.sub(r'#[\W]*',' ',x))

df

,id,website,Service
0,0,Gitlab.com,from project planning and source code manageme...
1,1,Cloudbees.com,"reduce risk, optimize software delivery and ac..."
2,2,squarespace.com,squarespace is the all-in-one solution for any...
3,3,Wix.com,create a free website with wix.com. choose a s...


In [4]:
df["Service"] = df["Service"].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
df['Service']    

0    from project planning and source code manageme...
1    reduce risk  optimize software delivery and ac...
2    squarespace is the all in one solution for any...
3    create a free website with wix com  choose a s...
Name: Service, dtype: object

In [5]:
#More data clearning
Appostophes = {"'s":"is","'re":"are"}
df['Service']
for i in range(len(df)):
  words =  df['Service'][i]
  for word in words:
    if word == "'s":
      word= "is"
    elif word == "'re":
      word =  "are"
  df['Service'][i] = words

df['Service']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0    from project planning and source code manageme...
1    reduce risk  optimize software delivery and ac...
2    squarespace is the all in one solution for any...
3    create a free website with wix com  choose a s...
Name: Service, dtype: object

In [6]:
documents = list(df['Service'])
print(documents)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

['from project planning and source code management to ci cd and monitoring  gitlab is a complete devops platform  delivered as a single application  only gitlab enables concurrent devops to make the software lifecycle 200  faster', 'reduce risk  optimize software delivery and accelerate innovation with cloudbees   the industry leading devops technology platform  build stuff that matters', 'squarespace is the all in one solution for anyone looking to create a beautiful website  domains  ecommerce  hosting  galleries  analytics  and 24 7 support all included', 'create a free website with wix com  choose a stunning template and customize anything with the wix website builder no coding skills needed  create yours today ']
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
#Extracting entities from text
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
def extract_entities(name, text):
    # Lopping over the sentences of the text
    for sent in nltk.sent_tokenize(text):
        # nltk.word_tokeize returns a list of words composing a sentence
        # nltk.pos_tag returns the position tag of words in the sentence
        # nltk.ne_chunk returns a label to each word based on this position tag when possible
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            try:
                if chunk.label() == 'PERSON':
                    for c in chunk.leaves():
                        if str(c[0].lower()) not in name:
                            name.append(str(c[0]).lower())
            except AttributeError:
                pass
    return name
## 
names = []
for doc in documents:
    names = extract_entities(names, doc)
## Update the stop words list
stop_words.update(names)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [0]:
#Stemming the similar words

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english") # Choose a language
texts = [[stemmer.stem(word) for word in document.lower().split() if (word not in stop_words)]
          for document in documents]

In [9]:
print(texts)

[['project', 'plan', 'sourc', 'code', 'manag', 'ci', 'cd', 'monitor', 'gitlab', 'complet', 'devop', 'platform', 'deliv', 'singl', 'applic', 'gitlab', 'enabl', 'concurr', 'devop', 'make', 'softwar', 'lifecycl', '200', 'faster'], ['reduc', 'risk', 'optim', 'softwar', 'deliveri', 'acceler', 'innov', 'cloudbe', 'industri', 'lead', 'devop', 'technolog', 'platform', 'build', 'stuff', 'matter'], ['squarespac', 'one', 'solut', 'anyon', 'look', 'creat', 'beauti', 'websit', 'domain', 'ecommerc', 'host', 'galleri', 'analyt', '24', '7', 'support', 'includ'], ['creat', 'free', 'websit', 'wix', 'com', 'choos', 'stun', 'templat', 'custom', 'anyth', 'wix', 'websit', 'builder', 'code', 'skill', 'need', 'creat', 'today']]


In [0]:
# Now the main part comes using gensim it will only create matrices based on unique words and if related words then it will treat as same both the words
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary(texts)

In [11]:
#Showing  the created dictionary
import itertools
list(itertools.islice(dictionary.token2id.items(), 0, 20))

[('200', 0),
 ('applic', 1),
 ('cd', 2),
 ('ci', 3),
 ('code', 4),
 ('complet', 5),
 ('concurr', 6),
 ('deliv', 7),
 ('devop', 8),
 ('enabl', 9),
 ('faster', 10),
 ('gitlab', 11),
 ('lifecycl', 12),
 ('make', 13),
 ('manag', 14),
 ('monitor', 15),
 ('plan', 16),
 ('platform', 17),
 ('project', 18),
 ('singl', 19)]

In [12]:
#creating corpus from the words in the dictionary
corpus = [dictionary.doc2bow(text) for text in texts]
corpus

#Here the first element in dict is the position of the  tag in the dictionary

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 2),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1)],
 [(8, 1),
  (17, 1),
  (20, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1)],
 [(35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1)],
 [(4, 1),
  (40, 2),
  (51, 2),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 2)]]

In [13]:
print(corpus[3][:20])

[(4, 1), (40, 2), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2)]


In [14]:
dictionary[48]

'solut'

In [15]:
#creating the tfidf for the created corpus 
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf

In [16]:
print(corpus_tfidf[3][:20])

[(4, 0.1203858530857692), (40, 0.2407717061715384), (51, 0.2407717061715384), (52, 0.2407717061715384), (53, 0.2407717061715384), (54, 0.2407717061715384), (55, 0.2407717061715384), (56, 0.2407717061715384), (57, 0.2407717061715384), (58, 0.2407717061715384), (59, 0.2407717061715384), (60, 0.2407717061715384), (61, 0.2407717061715384), (62, 0.2407717061715384), (63, 0.4815434123430768)]


In [0]:
#Converting the gensim matrix into sparse matrix to run the cosine similarities on them

import gensim
import numpy as np

corpus_tfidf_csr = gensim.matutils.corpus2csc(corpus_tfidf)
corpus_tfidf_numpy = corpus_tfidf_csr.T.toarray()

In [0]:
#Running the cosine similarities

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
cosine_similarities = linear_kernel(corpus_tfidf_numpy, corpus_tfidf_numpy) 
results = {}
idx = 0
row = 0
count = 0
for idx, row in df.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], df['id'][i]) for i in similar_indices] 
   results[row['id']] = similar_items[1:]

In [0]:
#Checking the results

def item(id):  
  return df.loc[df['id'] == id]['website'].tolist()[0].split(' - ')[0] 

def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")    
    recs = results[item_id][:num]   
    for rec in recs: 
       print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [20]:
#Recommendation results but the performance is low 

#Lets run the created model on bigger dataset
recommend(item_id=1, num=2)

Recommending 2 products similar to Cloudbees.com...
-------
Recommended: Gitlab.com (score:0.05654027941921757)
Recommended: Wix.com (score:0.0)


In [37]:
#Now running the model on g2 data

import numpy as np
import pandas as pd
df = pd.DataFrame()

id =  [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
website = ['Cloudflare','Amazon CloudFront','Fastly','CDN77','KeyCDN','Imperva Incapsula','Linodes Standard','Google Cloud CDN','Bootstrapcdn','StackPath CDN','Rackspace CDN','Microsoft Azure CDN','Akamai','Sucuri','Kollective','CDNify','Cachefly','Dexecure','IBM Enterprise Video Streaming','CDNetworks']

Service = ['What We Do At Cloudflare, we have our eyes set on an ambitious goal -- to help build a better Internet. We believe that with our talented team, smart technology and engaged users we can solve some of the biggest problems on the Internet','Amazon CloudFront is a web service for content delivery. It integrates with other Amazon Web Services to give developers and businesses an easy way to distribute content to end users with low latency, high data transfer speeds, and no commitments','Fastly helps the world’s most popular digital businesses keep pace with their customer expectations by delivering fast, secure, and scalable online experiences','CDN77 is a Content Delivery Network with 35+ points of presence around the world (incl. 3 in South America). CDN77 has its own Video Processing and Delivery platform called Streamflow','KeyCDN is a simple, fast & reliable Content Delivery Network with 25+ POPs strategicially located around the globe. KeyCDN offers the lowest price at $0.04/GB (every region), pay-as-you-go and many advanced features such as HTTP/2 Support, Free SSL, Instant Purging, a RESTful API, Real-time Analytics, Two-Factor Auth, and 24+ CMS integrations (WordPress, Drupal, Joomla, Magento, etc',"Imperva Incapsula delivers an enterprise-grade Web Application Firewall to safeguard your site from the latest threats, an intelligent and instantly effective 360-degree anti-DDoS solutions (layers 3-4 and 7), a global CDN to speed up your website's load speed and minimize bandwidth usage and an array of performance monitoring and analytic services to provide insights about your website's security and performance","Linode accelerates innovation by making cloud computing simple, accessible, and affordable to all. Founded in 2003, Linode helped pioneer the cloud computing industry and is today the largest independent open cloud provider in the world. Headquartered in Philadelphia's Old City, the company empowers more than a million developers, startups, and businesses across its global network of data centers",'Content Delivery Network for Cloud Platform Google Cloud CDN leverages Google\' s globally distributed edge points of presence to accelerate content delivery for websites and applications served out of Google Compute Engine and Google Cloud Storage. Cloud CDN lowers network latency, offloads origins, and reduces serving costs. Once you\'ve set up HTTP(S) Load Balancing, simply enable Cloud CDN with a single checkbox','Bootstrapcdn is the recommended CDN for Bootstrap, Font Awesome and Bootswatch','Innovative Content Delivery Network providing cutting-edge web services and info. like EdgeSSL and DualCDN Strategies','Accelerate your websites, web applications, and media delivery','A fast and modern global delivery network for high-bandwidth content','The Akamai Intelligent Platform is a cloud platform for delivering secure, high-performing user experiences to any device, anywhere. It reaches globally and delivers locally','Sucuri is a managed security service provider for websites. Our cloud-based tools provide complete website security solution, including performance optimization via a CDN, mitigation of external attacks like vulnerability exploits and DDoS attacks, and professional response in the event of security incident','At Kollective, we make corporate networks smarter so your people can work better. The Kollective platform scales existing IT infrastructure to accelerate content delivery to the edge of the enterprise while minimizing network congestion','CDNify is a flexible and easy-to-use content delivery network built for developers, startups, and digital agencies to accelerate your website, speed up load times and provide a faster, more reliable experience for your customers','Launched in 2002, CacheFly developed the world’s first TCP-anycast based Content Delivery Network—the only network built for throughput. From the first byte to the last byte, CacheFly delivers your files faster','We are passionate web performance experts that want to make the internet faster. Dexecure is the only automatic web performance solution that accelerates websites, for better business results. Dexecure automatically performs Front end optimizations and partners with Content Delivery Networks (CDN) to deliver the fastest loading site','IBM Watson Media (formerly IBM Cloud Video) provides an enterprise video solution, powered by Watson AI, for creating, storing, managing, broadcasting and measuring the impact of live and recorded video','CDNetworks enables Global Cloud Acceleration with 160 global points of presence including China, Russia and other emerging markets']

df['id'] = id
df['website'] = website
df['Service'] = Service

df

,id,website,Service
0,0,Cloudflare,"What We Do At Cloudflare, we have our eyes set..."
1,1,Amazon CloudFront,Amazon CloudFront is a web service for content...
2,2,Fastly,Fastly helps the world’s most popular digital ...
3,3,CDN77,CDN77 is a Content Delivery Network with 35+ p...
4,4,KeyCDN,"KeyCDN is a simple, fast & reliable Content De..."
5,5,Imperva Incapsula,Imperva Incapsula delivers an enterprise-grade...
6,6,Linodes Standard,Linode accelerates innovation by making cloud ...
7,7,Google Cloud CDN,Content Delivery Network for Cloud Platform Go...
8,8,Bootstrapcdn,Bootstrapcdn is the recommended CDN for Bootst...
9,9,StackPath CDN,Innovative Content Delivery Network providing ...


In [38]:
#Data cleaning steps
from html.parser import HTMLParser
html_parser = HTMLParser()
df['Service'] = df['Service'].apply(lambda x: html_parser.unescape(x))
df


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,id,website,Service
0,0,Cloudflare,"What We Do At Cloudflare, we have our eyes set..."
1,1,Amazon CloudFront,Amazon CloudFront is a web service for content...
2,2,Fastly,Fastly helps the world’s most popular digital ...
3,3,CDN77,CDN77 is a Content Delivery Network with 35+ p...
4,4,KeyCDN,"KeyCDN is a simple, fast & reliable Content De..."
5,5,Imperva Incapsula,Imperva Incapsula delivers an enterprise-grade...
6,6,Linodes Standard,Linode accelerates innovation by making cloud ...
7,7,Google Cloud CDN,Content Delivery Network for Cloud Platform Go...
8,8,Bootstrapcdn,Bootstrapcdn is the recommended CDN for Bootst...
9,9,StackPath CDN,Innovative Content Delivery Network providing ...


In [39]:
import re
df['Service']= df['Service'].apply(lambda x: x.lower())

#Extracting only words from Service
df['Service'] = df['Service'].apply(lambda x: re.sub(r'#[\W]*',' ',x))

df

,id,website,Service
0,0,Cloudflare,"what we do at cloudflare, we have our eyes set..."
1,1,Amazon CloudFront,amazon cloudfront is a web service for content...
2,2,Fastly,fastly helps the world’s most popular digital ...
3,3,CDN77,cdn77 is a content delivery network with 35+ p...
4,4,KeyCDN,"keycdn is a simple, fast & reliable content de..."
5,5,Imperva Incapsula,imperva incapsula delivers an enterprise-grade...
6,6,Linodes Standard,linode accelerates innovation by making cloud ...
7,7,Google Cloud CDN,content delivery network for cloud platform go...
8,8,Bootstrapcdn,bootstrapcdn is the recommended cdn for bootst...
9,9,StackPath CDN,innovative content delivery network providing ...


In [40]:
df["Service"] = df["Service"].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
df['Service']    

0     what we do at cloudflare  we have our eyes set...
1     amazon cloudfront is a web service for content...
2     fastly helps the world s most popular digital ...
3     cdn77 is a content delivery network with 35  p...
4     keycdn is a simple  fast   reliable content de...
5     imperva incapsula delivers an enterprise grade...
6     linode accelerates innovation by making cloud ...
7     content delivery network for cloud platform go...
8     bootstrapcdn is the recommended cdn for bootst...
9     innovative content delivery network providing ...
10    accelerate your websites  web applications  an...
11    a fast and modern global delivery network for ...
12    the akamai intelligent platform is a cloud pla...
13    sucuri is a managed security service provider ...
14    at kollective  we make corporate networks smar...
15    cdnify is a flexible and easy to use content d...
16    launched in 2002  cachefly developed the world...
17    we are passionate web performance experts 

In [41]:
Appostophes = {"'s":"is","'re":"are"}
df['Service']
for i in range(len(df)):
  words =  df['Service'][i]
  for word in words:
    if word == "'s":
      word= "is"
    elif word == "'re":
      word =  "are"
  df['Service'][i] = words

df['Service']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


0     what we do at cloudflare  we have our eyes set...
1     amazon cloudfront is a web service for content...
2     fastly helps the world s most popular digital ...
3     cdn77 is a content delivery network with 35  p...
4     keycdn is a simple  fast   reliable content de...
5     imperva incapsula delivers an enterprise grade...
6     linode accelerates innovation by making cloud ...
7     content delivery network for cloud platform go...
8     bootstrapcdn is the recommended cdn for bootst...
9     innovative content delivery network providing ...
10    accelerate your websites  web applications  an...
11    a fast and modern global delivery network for ...
12    the akamai intelligent platform is a cloud pla...
13    sucuri is a managed security service provider ...
14    at kollective  we make corporate networks smar...
15    cdnify is a flexible and easy to use content d...
16    launched in 2002  cachefly developed the world...
17    we are passionate web performance experts 

In [42]:
documents = list(df['Service'])
print(documents)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

['what we do at cloudflare  we have our eyes set on an ambitious goal    to help build a better internet  we believe that with our talented team  smart technology and engaged users we can solve some of the biggest problems on the internet', 'amazon cloudfront is a web service for content delivery  it integrates with other amazon web services to give developers and businesses an easy way to distribute content to end users with low latency  high data transfer speeds  and no commitments', 'fastly helps the world s most popular digital businesses keep pace with their customer expectations by delivering fast  secure  and scalable online experiences', 'cdn77 is a content delivery network with 35  points of presence around the world  incl  3 in south america   cdn77 has its own video processing and delivery platform called streamflow', 'keycdn is a simple  fast   reliable content delivery network with 25  pops strategicially located around the globe  keycdn offers the lowest price at  0 04 gb

In [43]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
def extract_entities(name, text):
    # Lopping over the sentences of the text
    for sent in nltk.sent_tokenize(text):
        # nltk.word_tokeize returns a list of words composing a sentence
        # nltk.pos_tag returns the position tag of words in the sentence
        # nltk.ne_chunk returns a label to each word based on this position tag when possible
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            try:
                if chunk.label() == 'PERSON':
                    for c in chunk.leaves():
                        if str(c[0].lower()) not in name:
                            name.append(str(c[0]).lower())
            except AttributeError:
                pass
    return name
## 
names = []
for doc in documents:
    names = extract_entities(names, doc)
## Update the stop words list
stop_words.update(names)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [44]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")
texts = [[stemmer.stem(word) for word in document.lower().split() if (word not in stop_words)]
          for document in documents]

print(texts)

[['cloudflar', 'eye', 'set', 'ambiti', 'goal', 'help', 'build', 'better', 'internet', 'believ', 'talent', 'team', 'smart', 'technolog', 'engag', 'user', 'solv', 'biggest', 'problem', 'internet'], ['amazon', 'cloudfront', 'web', 'servic', 'content', 'deliveri', 'integr', 'amazon', 'web', 'servic', 'give', 'develop', 'busi', 'easi', 'way', 'distribut', 'content', 'end', 'user', 'low', 'latenc', 'high', 'data', 'transfer', 'speed', 'commit'], ['fast', 'help', 'world', 'popular', 'digit', 'busi', 'keep', 'pace', 'custom', 'expect', 'deliv', 'fast', 'secur', 'scalabl', 'onlin', 'experi'], ['cdn77', 'content', 'deliveri', 'network', '35', 'point', 'presenc', 'around', 'world', 'incl', '3', 'south', 'america', 'cdn77', 'video', 'process', 'deliveri', 'platform', 'call', 'streamflow'], ['keycdn', 'simpl', 'fast', 'reliabl', 'content', 'deliveri', 'network', '25', 'pop', 'strategici', 'locat', 'around', 'globe', 'keycdn', 'offer', 'lowest', 'price', '0', '04', 'gb', 'everi', 'region', 'pay', 'g

In [0]:
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary(texts)


In [46]:
import itertools
list(itertools.islice(dictionary.token2id.items(), 0, 20))


[('ambiti', 0),
 ('believ', 1),
 ('better', 2),
 ('biggest', 3),
 ('build', 4),
 ('cloudflar', 5),
 ('engag', 6),
 ('eye', 7),
 ('goal', 8),
 ('help', 9),
 ('internet', 10),
 ('problem', 11),
 ('set', 12),
 ('smart', 13),
 ('solv', 14),
 ('talent', 15),
 ('team', 16),
 ('technolog', 17),
 ('user', 18),
 ('amazon', 19)]

In [47]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 2),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1)],
 [(18, 1),
  (19, 2),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 2),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 2),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 2)],
 [(9, 1),
  (20, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 2),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1)],
 [(23, 1),
  (25, 2),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 2),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1)],
 [(23, 1),
  (25, 1),
  (32, 1),
  (45, 1),
  (56, 1),
  (60, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79,

In [48]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

corpus_tfidf

In [0]:
import gensim
import numpy as np

corpus_tfidf_csr = gensim.matutils.corpus2csc(corpus_tfidf)
corpus_tfidf_numpy = corpus_tfidf_csr.T.toarray()

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
cosine_similarities = linear_kernel(corpus_tfidf_numpy, corpus_tfidf_numpy) 
results = {}
idx = 0
row = 0
count = 0
for idx, row in df.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
   similar_items = [(cosine_similarities[idx][i], df['id'][i]) for i in similar_indices] 
   results[row['id']] = similar_items[1:]

In [0]:
#Checking the results

def item(id):  
  return df.loc[df['id'] == id]['website'].tolist()[0].split(' - ')[0] 

def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")    
    recs = results[item_id][:num]   
    for rec in recs: 
       print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [52]:
#Recommendations results again the performance is not that much good but still it works
recommend(item_id=1,num=5)

Recommending 5 products similar to Amazon CloudFront...
-------
Recommended: CDNify (score:0.1049838103636187)
Recommended: StackPath CDN (score:0.10105379012733819)
Recommended: Dexecure (score:0.09212860316978821)
Recommended: Rackspace CDN (score:0.0902306567126283)
Recommended: Imperva Incapsula (score:0.07996080297704466)
